<a href="https://colab.research.google.com/github/devps814/Summer-Analytics-W-1/blob/main/Summer_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#📦 Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, f1_score


In [ ]:

# 📥 Step 2: Load Data
train = pd.read_csv('/content/Train_Data.csv')
test = pd.read_csv('/content/Test_Data.csv')
sample_submission = pd.read_csv('/content/Sample_Submission.csv')


In [ ]:
# 🔍 Clean age_group column
train['age_group'] = train['age_group'].astype(str).str.strip()

In [ ]:
# 🧹 Handle categorical columns with 7/9 (assumed invalid/missing)
categorical_cols = ['RIAGENDR', 'PAQ605', 'DIQ010']
for col in categorical_cols:
    train[col] = train[col].replace([7, 9], np.nan)
    test[col] = test[col].replace([7, 9], np.nan)
    train[col].fillna(train[col].mode()[0], inplace=True)
    test[col].fillna(train[col].mode()[0], inplace=True)
    train[col] = train[col].astype(int)
    test[col] = test[col].astype(int)

/tmp/ipython-input-58-3049235174.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(train[col].mode()[0], inplace=True)
/tmp/ipython-input-58-3049235174.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tru

In [ ]:
# 🔄 Encode target variable: Adult=0, Senior=1
train['age_group'] = train['age_group'].astype(str).str.strip().replace({'Adult': 0, 'Senior': 1})
train['age_group'] = pd.to_numeric(train['age_group'], errors='coerce').fillna(0).astype(int)



In [ ]:
# 🔍 Fill numeric missing values with mean
numeric_cols = train.select_dtypes(include=np.number).columns.tolist()
numeric_cols.remove('age_group')  # exclude target

In [ ]:
for col in numeric_cols:
    train[col].fillna(train[col].mean(), inplace=True)
    test[col].fillna(train[col].mean(), inplace=True)



/tmp/ipython-input-64-33252035.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(train[col].mean(), inplace=True)
/tmp/ipython-input-64-33252035.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [ ]:
# 🧪 Feature Engineering
train['glucose_bmi'] = train['LBXGLU'] / (train['BMXBMI'] + 1)
test['glucose_bmi'] = test['LBXGLU'] / (test['BMXBMI'] + 1)
train['insulin_ratio'] = train['LBXIN'] / (train['LBXGLU'] + 1)
test['insulin_ratio'] = test['LBXIN'] / (test['LBXGLU'] + 1)
train['is_active'] = (train['PAQ605'] == 1).astype(int)
test['is_active'] = (test['PAQ605'] == 1).astype(int)

In [ ]:
# 🔢 Extract test IDs and remove from features
if 'SEQN' in test.columns:
    test_ids = test['SEQN']
    test.drop(columns=['SEQN'], inplace=True)
if 'SEQN' in train.columns:
    train.drop(columns=['SEQN'], inplace=True)

In [ ]:
# 🔢 Features and Target
X = train.drop('age_group', axis=1)
y = train['age_group']

In [ ]:
# 🔄 Scale Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_scaled = scaler.transform(test)



In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)



In [ ]:
# 🚀 Train XGBoost Model
model = XGBClassifier(n_estimators=300, learning_rate=0.05, max_depth=4, random_state=42, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:05:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=300,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [ ]:
# 📊 Evaluate
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))
print("F1 Score:", f1_score(y_val, y_pred))


              precision    recall  f1-score   support

           0       0.85      0.98      0.91       331
           1       0.47      0.11      0.18        63

    accuracy                           0.84       394
   macro avg       0.66      0.54      0.54       394
weighted avg       0.79      0.84      0.79       394

F1 Score: 0.1794871794871795


In [ ]:
test_preds = model.predict(test_scaled)

In [ ]:
# 📄 Save Final Submission in /content/
final_submission = pd.DataFrame({'age_group': test_preds})
final_submission.to_csv('/content/final_submission.csv', index=False)
print("✅ Final submission file saved as 'final_submission.csv'")



✅ Final submission file saved as 'final_submission.csv'
